In [ ]:
!pip install matplotlib
!pip install numpy
# !pip install plotly
!pip install pickle-mixin
# !pip install pytest-shutil

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from XrayLib import xray

with open('xrayobj.pkl', 'rb') as pickle_file:
    a = pickle.load(pickle_file)
    
a.plottiff(show=1)
a.plotreflectivity(showreflect=1)

In [ ]:
import glob
FitResults = glob.glob("./*.fit")

plt.figure()
for Result in FitResults:
    FitData = np.genfromtxt(Result, delimiter='\t', skip_header=31)
    p_tmp = plt.plot(FitData[:,0],FitData[:,1],'.')
    plt.plot(FitData[:,0],FitData[:,3],'-',color=p_tmp[0].get_color())
    plt.ylabel('Normalized Intensity')
    plt.xlabel('q parrallel (Å^-1)')